# Assignment 1


In [2]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division


C:\Users\s134277\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\s134277\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [82]:
#create co-occurrence matrix
shape = (V, V)
matrix = np.zeros(shape)
#create the co-occurence matrix, checking for every word only the words in the window size with a higher index
for sentence in corpus:
    for i in range(0, len(sentence)):
        coword = sentence[i+1:i+window_size_corpus+1]
        for j in coword:
            matrix[sentence[i], j] = matrix[sentence[i], j] + 1
#Add the transposed matrix to the current matrix to includes words that have index[i-window_size_corpus, i-1]
matrix_trans = matrix.transpose()
co_occurence_matrix = np.add(matrix, matrix_trans)

In [83]:
#normalize the words by dividing the co-occurence frequency with the amount of words
for i in range(1, V):
    for j in range(1,V):
        co_occurence_matrix[j,i] = co_occurence_matrix[j,i]/V

#find cosine similarity to Alice, Dinah and Rabbit
alice_index = tokenizer.word_index['alice']
dinah_index = tokenizer.word_index['dinah']
rabit_index = tokenizer.word_index['rabbit']
#create similarity matrix
sim = cosine_similarity(co_occurence_matrix)
sim_alice_dinah = sim[alice_index, dinah_index]
sim_alice_rabit = sim[alice_index, rabit_index]
sim_dinah_rabit = sim[dinah_index, rabit_index]
print("Cosine similarity Alice - Dinah: %.4f" %sim_alice_dinah)
print("Cosine similarity Alice - Rabbit: %.4f" %sim_alice_rabit)
print("Cosine similarity Dinah - Rabbit: %.4f" %sim_dinah_rabit)

Cosine similarity Alice - Dinah: 0.3936
Cosine similarity Alice - Rabbit: 0.4789
Cosine similarity Dinah - Rabbit: 0.2986


In [103]:
#find the closest words to Alice

# The closest word to 'alice' using nn is 'alice'. We search the five closest neighbours to alice that are not 
# identical to 'alice'. Therefore we set the number of neighbours to be 6 and we only look at the last five.

#we fit the nearest neighbors to the co-occurence matrix, but not to the first column, as this one is empty.
nbrs = nn(n_neighbors=6, algorithm='auto').fit(co_occurence_matrix[1:,])
distances, indices = nbrs.kneighbors(co_occurence_matrix[1:,])
# index(co_occurence_matrix = the index(co_occurence_matrix[1:,]) +1
print(indices[alice_index-1])
test = np.add(indices[alice_index-1],1)
print(test)
for word, index in tokenizer.word_index.items():    # for name, age in list.items():  (for Python 3.x)
    if tokenizer.word_index[word] in test:
        print(word)
      

[10 13 40 12 17 20]
[11 14 41 13 18 21]
alice
that
her
for
on
herself
[11 31 61 32 48 43]
[11 14 41 13 18 21]
alice
that
her
for
on
herself


In [1]:
indices

NameError: name 'indices' is not defined

In [101]:
print(indices[alice_index])
print(indices[alice_index-1])
print(np.add(indices[alice_index-1],1))

[11 31 61 32 48 43]
[10 13 40 12 17 20]
[11 14 41 13 18 21]


In [73]:
print('The length of the co-occurence matrix is {}'.format(len(co_occurence_matrix)))

The length of the co-occurence matrix is 1183


Trivially, the closest word to Alice is Alice. However, all other works are not in line with what one would expect. Having read the first lines of Alice in Wonderland, it is for example not clear what the connection is between 'Alice' and 'foot'. A possible explanation is that the dimensionality of the data is high, as the co-occurency of two terms is compared with all (1182-2=)1180. Because of this high dimensionality, the distance difference between the nearest and farthest neighbor is low. To illustrate this, we computed the 100 nearest neighbours to 'Alice' below and compared the longest of these 100 distances with the smallest distance bigger than zero. As can be seen in the output of the code below, the nearest neighbour is only 1.9% closer in terms of distance than the 99th nearest neighbour.

In [74]:
nbrs_100 = nn(n_neighbors=1000, algorithm='auto').fit(co_occurence_matrix)
distances_1000, indices_1000 = nbrs.kneighbors(co_occurence_matrix)

print(distances[alice_index]/np.average(distances_1000[alice_index]))
print(distances[alice_index]/np.max(distances_1000[alice_index]))
np.max(np.max(distances_1000[alice_index])/distances[alice_index][1:])

[0.         1.18444348 1.19708691 1.2047711  1.20678027 1.20691825]
[0.         0.98137838 0.99185417 0.99822096 0.99988567 1.        ]


1.018974966476399

In [8]:
#percentage of cells with zeros
print('The ratio of cells with zeros is {:.4f}'.format
      (np.where(co_occurence_matrix==0)[0].size/co_occurence_matrix.size))

The ratio of cells with zeros is 0.9816


Discussion of the drawback of a term-term co-occurence matrix:
- A co-occurence matrix is very sparse. In this data set, 98,16% of the cells in the word-word co-occurence matrix are zeros. 
- The matrix tends to be very large. For the first 700 lines of the book Alice in Wonderland, we already need a matrix of size 1183x1183. 
- The frequency of occurence can be skewed and non-discriminative. Words like 'the', 'and' and 'to' are not very discriminative, but the impact on the calculation of word similarity of these words is equally big as the impact of more discriminative words. However, this is not necessary a drawback of the use of the word-word co-occurence matrix, as one can adjust for this by using weighting terms.

In [104]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = [co_occurence_matrix][:,1:]
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    #Dit werkt wel, maar dan krijg je later weer problemen
#     f.write(" ".join(map(str, list(vectors[0][i,:]))))
    f.write("\n")
f.close()

#reopen your file as follows
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

TypeError: list indices must be integers or slices, not tuple

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [ ]:
#prepare data for cbow



In [ ]:
#create CBOW model


In [ ]:
#define loss function


In [ ]:
#train model


In [61]:
#generate data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(sentence[i])
            if in_words != []:
                x = np.array(in_words,dtype=np.int32)
                y = np_utils.to_categorical(labels, V)
                yield (x,y)


In [56]:
#create Skipgram model
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))


In [57]:
#define loss function for Skipgram
skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [62]:
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram.train_on_batch(x, y)

    print(ite, loss)

0 14465.41867351532


KeyboardInterrupt: 

In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer
skipgram_al = Sequential()
skipgram_al.add(Embedding(input_dim=V, output_dim=dim, init='glorot_uniform', input_length=1))
skipgram_al.add(Reshape((dim, )))
skipgram_al.add(Dense(input_dim = dim, output_dim = dim, activation = 'linear'))
skipgram_al.add(Dense(input_dim=dim, output_dim=V, activation='softmax'))


In [ ]:
#define loss function for Skipgram + dense
skipgram_al.compile(loss='categorical_crossentropy', optimizer="sgd")


In [ ]:
#train model for Skipgram + dense

for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += skipgram_al.train_on_batch(x, y)

    print(ite, loss)

In [67]:
#Implement your own analogy function

vectors = skipgram.get_weights()[0]
vectors

array([[-1.3474584e-02, -2.6088379e-02, -2.1072652e-02, ...,
        -9.9375844e-05, -4.3099836e-02, -1.1925038e-02],
       [ 7.0244052e-02,  1.4387365e-01, -8.1885554e-02, ...,
        -1.3903263e-01,  1.3084920e-01,  2.7086593e-02],
       [ 2.2625022e-02, -6.4422481e-02,  1.3710096e-02, ...,
        -1.9175114e-02,  7.5223371e-02, -4.3790504e-02],
       ...,
       [-5.3781688e-02,  3.5365075e-02,  3.3549853e-02, ...,
        -4.3081380e-02, -7.8925900e-03,  4.0033832e-02],
       [-3.5017531e-02, -1.0825522e-02, -6.5966025e-02, ...,
        -4.4772923e-02, -6.5754987e-02,  4.0927239e-02],
       [-3.9067667e-02,  4.7268100e-02, -6.0884938e-02, ...,
         3.9025098e-02, -6.7781709e-02, -1.1597060e-02]], dtype=float32)

Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings